In [0]:
!pip install cupy
# Установка библиотеки cupy
# Необязательна, если встроена в коллаб
# Клетку можно спокойно пропустить

  Using cached cupy-7.5.0.tar.gz (3.7 MB)


    ERROR: Command errored out with exit status 1:
     command: 'C:\Users\taran\anaconda3\python.exe' -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\taran\\AppData\\Local\\Temp\\pip-install-bsy3wk95\\cupy\\setup.py'"'"'; __file__='"'"'C:\\Users\\taran\\AppData\\Local\\Temp\\pip-install-bsy3wk95\\cupy\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' egg_info --egg-base 'C:\Users\taran\AppData\Local\Temp\pip-install-bsy3wk95\cupy\pip-egg-info'
         cwd: C:\Users\taran\AppData\Local\Temp\pip-install-bsy3wk95\cupy\
    Complete output (43 lines):
    Options: {'package_name': 'cupy', 'long_description': None, 'wheel_libs': [], 'wheel_includes': [], 'no_rpath': False, 'profile': False, 'linetrace': False, 'annotate': False, 'no_cuda': False, 'use_hip': False}
    
    -------- Configuring Module: cuda --------
    Microsoft Visual C++ 14.0 

In [0]:
import cupy as cp
import random


In [0]:
# Функции, которые нам потребуются
def draw_pole(pole):
  # Отрисовывает поле
    for i in pole:
        temp = [int(j) if j > 0 else ' ' for j in i]
        print((f'+----+----+----+----+\n|'
                f'{temp[0]:4}|{temp[1]:4}|{temp[2]:4}|{temp[3]:4}|'))
    print('+----+----+----+----+')


def new_value(pole):
  # Вставляет значение на случайную пустую клетку. 
    #в игре закидывается либо 2 либо 4
    empty = []
    for i in range(4):
        for j in range(4):
            if pole[i, j] == 0:
                empty.append((i, j))
    x, y = random.choice(empty)
    prob = random.random()
    if prob <= 0.9:
        pole[x, y] = 2
         # 2 вставляется на пустую клетку с вероятностью 90 процентов
    else:
        pole[x, y] = 4
        # 4 вставляется с вероятность 10 процентов


def check(pole):
    # Проверяет, есть ли хоть 1 возможный ход
    for i in range(4):
        for j in range(4):
            if pole[i, j] == 0:
                return True
                # Если есть пустые ячейки, значит можно что-то куда-то сдвинуть
    for i in range(3):
        for j in range(3):
            if pole[i, j] == pole[i + 1, j] or pole[i, j] == pole[i, j + 1]:
                return True
                # Если есть 2 рядом стоящих одинаковых значения, значит тоже хоть один ход еще есть
    return False
    # Иначе ошибка, конец игры


def shift(pole):
  # Сдвигает все влево: либо сдвигает элементы на пустые поля, либо складывает одинаковые элементы в строчке
    count = 0
    score = 0
    for i in range(4):
        for j in range(1, 4):
            for k in range(j, 0, -1):
                if pole[i, k-1] == 0 or pole[i, k-1] == pole[i, k]:
                    if pole[i, k-1] == pole[i, k]:
                        score += pole[i, k]
                    pole[i, k-1] += pole[i, k]
                    pole[i, k] = 0
                    count += 1
    return count, score


def make_step(pole, step: int):
   # Делает сдвиг. принимает 2 аргумента: поле и саму команду
    # Шаг влево
    if step == 1:
        t = shift(pole)
        return t
    # Шаг вверх
    elif step == 2:
        pole = cp.rot90(pole, k=1)
        # Переворачиваем матрицу на 90 градусов, вызываем шифт влево, переворачиваем матрицу обратно
        t = shift(pole)
        pole = cp.rot90(pole, k=3)
        return t
    # Шаг вправо
    elif step == 3:
        pole = cp.rot90(pole, k=2)
        # Аналогично шагу вверх
        t = shift(pole)
        pole = cp.rot90(pole, k=2)
        return t
    # Шаг вниз
    else:
        pole = cp.rot90(pole, k=3)
        # Аналогично шагу вверх
        t = shift(pole)
        pole = cp.rot90(pole, k=1)
        return t


In [0]:
a = cp.zeros((4, 4), dtype='int32')
# Создается двумерный массив размером 4*4
new_value(a)
# Вставляется одно случайное значение
scores = 0
# Задается счет, изначально равный 0
while (check(a)):
    print(f'Score: {scores}')
    # Перед каждым ходом выводим количество набранных очков
    draw_pole(a)
    # Выводим поле
    com = int(input())
    # Ожидаем ввода команды
    steps, score = make_step(a, com)
    # Считает количество перемещений и количество очков, которые мы получили за этот ход
    scores += score
    # Количество очков прибавляется к изначальному до хода
    if steps == 0:
      # Надо посмотреть произошло ли перемещение, иначе не происходит вставка нового элемента
        continue
    new_value(a)


Score: 0
+----+----+----+----+
|   2|    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
|    |    |    |    |
+----+----+----+----+
